In [ ]:
import os
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed
from modules.StrategySearcher import StrategySearcher
import warnings
warnings.filterwarnings("ignore")

label_methods = [
    "atr",
    # "trend",
    # "trend_profit",
    # "trend_multi",
    # "clusters",
    # "multi_window",
    # "validated_levels",
    # "zigzag",
    # "mean_rev",
    # "mean_rev_multi",
    # "mean_rev_vol",
    # "filter",
    # "multi_filter",
    # "filter_bidirectional",
    # "filter_one",
    # "trend_one",
    # "filter_flat",
]

base_cfg = dict(
    symbol='XAUUSD', timeframe='H1', direction='both',
    train_start=datetime(2019,1,1), train_end=datetime(2025,1,1),
    test_start=datetime(2021,1,1),  test_end=datetime(2022,1,1),
    search_type='lgmm', n_trials=1000, n_models=1
)

configs = []
for lm in label_methods:
    cfg = base_cfg.copy()
    cfg["label_method"] = lm
    configs.append(cfg)

# Calcular núcleos por configuración
# total_cores = os.cpu_count()
# cores_per_config = max(1, total_cores // len(configs))

# Actualizar n_jobs en cada configuración
for cfg in configs:
    #cfg["n_jobs"] = cores_per_config
    cfg["tag"] = f"{cfg['symbol']} {cfg['timeframe']} {cfg['direction']} {cfg['label_method']} {cfg['search_type']} {cfg.get('search_subtype', '')}".strip()

def launch(cfg):
    s = StrategySearcher(**cfg)
    s.run_search()

with ProcessPoolExecutor(max_workers=len(configs)) as pool:
    futures = {pool.submit(launch, c): c["tag"] for c in configs}
    for f in as_completed(futures):
        tag = futures[f]
        try:
            print(f"[{tag}] terminado")
        except Exception as e:
            print(f"[{tag}] falló: {e}")

[XAUUSD H1 both trend lgmm] modelo 0 trial 1/1000 ins=-1.000000 oos=-1.000000 avg=8.34s mem=413.41MB check_constant_features:0.00MB get_train_test_data:178.18MB _apply_labeling:4.36MB fit_final_models:0.00MB _apply_labeling:0.00MB fit_final_models:0.00MB _evaluate_clusters:4.36MB search_lgmm:186.02MB
[XAUUSD H1 both atr lgmm] modelo 0 trial 1/1000 ins=0.166198 oos=-1.000000 avg=18.22s mem=822.15MB check_constant_features:0.00MB get_train_test_data:198.97MB _apply_labeling:0.00MB fit_final_models:297.62MB _apply_labeling:0.00MB fit_final_models:0.00MB _apply_labeling:0.00MB fit_final_models:105.87MB _apply_labeling:0.00MB fit_final_models:0.00MB _apply_labeling:0.00MB fit_final_models:0.00MB _apply_labeling:0.00MB fit_final_models:0.00MB _apply_labeling:0.00MB fit_final_models:0.00MB _apply_labeling:0.00MB fit_final_models:41.48MB _evaluate_clusters:440.69MB search_lgmm:596.23MB
[XAUUSD H1 both atr lgmm] modelo 0 trial 2/1000 ins=0.047306 oos=0.010516 avg=16.31s mem=997.97MB check_const